In [0]:
import os
import tensorflow as tf
import time
import numpy as np

In [3]:
def add_op(x, y):
  return x + y

x = tf.placeholder(tf.float32, [100,])
y = tf.placeholder(tf.float32, [100,])

session = tf.Session()

with tf.Session() as session:
  start = time.time()
  result = session.run(add_op(x,y), feed_dict={x:np.arange(100), y:np.arange(100)})
  end = time.time()
  elapsed = end-start
  print(elapsed)
  print(result)

0.018573522567749023
[  0.   2.   4.   6.   8.  10.  12.  14.  16.  18.  20.  22.  24.  26.
  28.  30.  32.  34.  36.  38.  40.  42.  44.  46.  48.  50.  52.  54.
  56.  58.  60.  62.  64.  66.  68.  70.  72.  74.  76.  78.  80.  82.
  84.  86.  88.  90.  92.  94.  96.  98. 100. 102. 104. 106. 108. 110.
 112. 114. 116. 118. 120. 122. 124. 126. 128. 130. 132. 134. 136. 138.
 140. 142. 144. 146. 148. 150. 152. 154. 156. 158. 160. 162. 164. 166.
 168. 170. 172. 174. 176. 178. 180. 182. 184. 186. 188. 190. 192. 194.
 196. 198.]


In [2]:
tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU Adrress is:', tpu_address)

with tf.Session(tpu_address) as session:
  devices = session.list_devices()
  
print('TPU Devices: ', devices)

TPU Adrress is: grpc://10.92.141.242:8470
TPU Devices:  [_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 4202958407359378302), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 457445209313220937), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 15629492005101444973), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 14668869961732076666), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 16808241019051115994), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 9973309496827891006), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 14845422368589526680), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 11318821564096090345), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 702677718

In [3]:
def add_op(x, y):
  return x + y

x = tf.placeholder(tf.float32, [100,])
y = tf.placeholder(tf.float32, [100,])

tpu_ops = tf.contrib.tpu.rewrite(add_op,[x,y])

session = tf.Session(tpu_address)

try:
  session.run(tf.contrib.tpu.initialize_system())
  start = time.time()
  print(session.run(tpu_ops, {x: np.arange(100), y:np.arange(100)}))
  end = time.time()
  elapsed = end - start
  print(elapsed)
finally:
  session.run(tf.contrib.tpu.shutdown_system())
  session.close()

[array([  0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,  18.,  20.,
        22.,  24.,  26.,  28.,  30.,  32.,  34.,  36.,  38.,  40.,  42.,
        44.,  46.,  48.,  50.,  52.,  54.,  56.,  58.,  60.,  62.,  64.,
        66.,  68.,  70.,  72.,  74.,  76.,  78.,  80.,  82.,  84.,  86.,
        88.,  90.,  92.,  94.,  96.,  98., 100., 102., 104., 106., 108.,
       110., 112., 114., 116., 118., 120., 122., 124., 126., 128., 130.,
       132., 134., 136., 138., 140., 142., 144., 146., 148., 150., 152.,
       154., 156., 158., 160., 162., 164., 166., 168., 170., 172., 174.,
       176., 178., 180., 182., 184., 186., 188., 190., 192., 194., 196.,
       198.], dtype=float32)]
0.022487640380859375


In [10]:
import tensorflow as tf
import keras
import tensorflow.keras.backend as K
import numpy as np
from tensorflow.contrib.tpu.python.tpu import keras_support
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, AveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
import os

def basic_mlp_module(input, units):
    x = Dense(units)(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)
    return x

def create_mlp_model():
    input = Input((32*32*3,))
    x = basic_mlp_module(input, 2048)
    x = basic_mlp_module(x, 1024)
    x = basic_mlp_module(x, 512)
    x = basic_mlp_module(x, 256)
    x = basic_mlp_module(x, 128)
    x = basic_mlp_module(x, 64)
    x = basic_mlp_module(x, 32)
    x = basic_mlp_module(x, 16)
    x = Dense(10, activation="softmax")(x)
    return Model(input, x)

def main():
    # 7 lets make this into TPU Code!!!
    K.clear_session()

    # CIFAR
    (X_train, y_train), (_, _) = cifar10.load_data()
    X_train = (X_train / 255.0).reshape(50000, -1)
    y_train = to_categorical(y_train)

    # Model building
    model = create_mlp_model()
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
    tpu_optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)
    model.compile(tpu_optimizer, loss="categorical_crossentropy", metrics=["acc"])
    
    tpu_grpc_url = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu_grpc_url)
    strategy = tf.contrib.tpu.TPUDistributionStrategy(tpu_cluster_resolver)
    model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=strategy)
  
    model.fit(X_train, y_train, batch_size=1024, epochs=10)

if __name__ == "__main__":
    main()

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.92.141.242:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 4202958407359378302)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 457445209313220937)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 15629492005101444973)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 14668869961732076666)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 16808241019051115994)
INFO:tensorflow:*** Available Device: _Devic